# Improvements after seeing others' Notebooks

Inspiration largely from Chong Zhen Jie:
- notebook - https://www.kaggle.com/code/chongzhenjie/ecuador-store-sales-global-forecasting-lightgbm/notebook#3.-Model-Training
- eda takeaways (not used here)
    * missed zero vals (leading, trailing 0s, number of 0s)
    * holidays (filtering for region, work days, etc.)
    * store clustering

- modelling takeaways
    * interpolate missing dates (like jan. 01, christmas)
    * dif model for each family
    * different samples of training data --> ensemble models

## What are the main improvements I can make?

- features
    * oil, oil rolling
    * transactions
    * store cluster, type, etc.

    * holidays merged (city, state, national, work) - or maybe we can delete holidays

- modelling
    * train dif model per family
    * zero out predictions if there are some # of trailing 0s
    * train models on dif train periods (can also exclude before 2015), then ensemble
    * try weighted lgbm highlighting end of aug (own idea, not sure if itll help at all)

In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, mean_squared_error
import lightgbm as lgb
from itertools import product
import re

In [57]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import OrdinalEncoder

In [3]:
train_df = pd.read_csv('data/train.csv', parse_dates=['date']).drop(columns='id')
test_df = pd.read_csv('data/test.csv', parse_dates=['date']).drop(columns='id')
transactions_df = pd.read_csv('data/transactions.csv', parse_dates=['date'])
oil_df = pd.read_csv('data/oil.csv', parse_dates=['date'])
holidays_df = pd.read_csv('data/holidays_events.csv', parse_dates=['date'])
stores_df = pd.read_csv('data/stores.csv')

## 1. Preprocessing (fix missing vals)

In [4]:
# Fix missing dates / NaNs in oil df
start_day = oil_df['date'].min()
end_day = oil_df['date'].max()
date_range = pd.date_range(start_day, end_day)
print(f"Days in Oil Range: {start_day.strftime('%Y-%m-%d')} to {end_day.strftime('%Y-%m-%d')}, {len(date_range)} days")
print(f"Days in Actual oil df: {len(oil_df['date'])}")

oil_all_days = oil_df.merge(pd.DataFrame({'date': date_range}), how='outer', on='date')
interpolated_oil = oil_all_days.interpolate(method='linear', limit_direction='both')
print(f"After interpolating, {interpolated_oil.isna().sum().sum()} missing values")

Days in Oil Range: 2013-01-01 to 2017-08-31, 1704 days
Days in Actual oil df: 1218
After interpolating, 0 missing values


In [5]:
# Delete all holidays in train
print("Train # Days:", len(train_df['date'].unique()))
print("# Holidays:", len(holidays_df['date'].unique()))

holidays_removed = train_df[~train_df['date'].isin(holidays_df['date'])]
holidays_removed.isna().sum()

print("Leftover days:", len(holidays_removed['date'].unique()))

Train # Days: 1684
# Holidays: 312
Leftover days: 1432


In [6]:
# Fix transactions df
filtered_transactions = transactions_df[transactions_df['date'].isin(holidays_removed['date'])]

num_days = len(holidays_removed['date'].unique())
num_stores = len(holidays_removed['store_nbr'].unique())
print("Days in train:", num_days)
print("Stores in train:", num_stores)
print("Expected # transactions:", num_days * num_stores)
print("Len Transactions:", len(filtered_transactions))

train_zero_sales = (holidays_removed.groupby(by=['date', 'store_nbr'])['sales'].sum() == 0).sum()
print("Stores w/ 0 sales in train:", train_zero_sales)
print("Not zero missing in transactions (unexpected):", num_days * num_stores - len(filtered_transactions) - train_zero_sales)
print()

# other way of getting days where a store had zero sales
# store_pivot = holidays_removed.pivot_table(index='date', columns='store_nbr', values='sales', aggfunc='sum')
# (store_pivot == 0).sum().sum()

zero_sales = (holidays_removed.groupby(by=['date', 'store_nbr'])['sales'].sum() == 0).to_frame('zero_sales').reset_index()
zero_sales

# Make transactions df have all days; some 0
all_days = pd.DataFrame(product(holidays_removed['date'].unique(), holidays_removed['store_nbr'].unique()), columns=['date', 'store_nbr'])
all_transactions = filtered_transactions.merge(all_days, on=['date', 'store_nbr'], how='outer')
print("Before na:", all_transactions.isna().sum().sum())
print("Zero sales:", len(zero_sales[zero_sales['zero_sales'] == True]))
all_transactions.loc[zero_sales['zero_sales'] == True,'transactions'] = 0
print("Remaining na:", all_transactions.isna().sum().sum())

# Fill the other NA Values with interpolation (b/c of graphs below)
final_transactions = all_transactions.groupby(by='store_nbr', group_keys=False).apply(lambda col : col.interpolate(method='linear')).sort_values(by=['date', 'store_nbr'])

# Check the other transactions
# why_nan = all_transactions[all_transactions.isna().sum(axis=1).astype(bool)].copy()
# why_nan['transactions'] = 1

# viz_transactions = holidays_removed.merge(why_nan, on=['date', 'store_nbr'], how='left')
# viz_transactions = viz_transactions.fillna(0)

# avg_sales = viz_transactions.groupby(by=['store_nbr', 'family', 'transactions'])['sales'].mean().to_frame('mean_sales').reset_index()
# avg_sales['mean_sales'] = np.log1p(avg_sales['mean_sales'])
# sns.boxplot(data=avg_sales, x='family', y='mean_sales', hue='transactions')
# plt.xticks(rotation=90);

Days in train: 1432
Stores in train: 54
Expected # transactions: 77328
Len Transactions: 71096
Stores w/ 0 sales in train: 6115
Not zero missing in transactions (unexpected): 117

Before na: 6232
Zero sales: 6115
Remaining na: 117


/var/folders/_c/j62lsd3n7x9_vq272kdsl3fc0000gn/T/ipykernel_43273/4119929746.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_transactions = all_transactions.groupby(by='store_nbr', group_keys=False).apply(lambda col : col.interpolate(method='linear')).sort_values(by=['date', 'store_nbr'])


In [58]:
""" 
What features do I add:
    1. oil
    2. transactions
    3. onpromotion
    rolling for all of the above
"""

combined = holidays_removed.copy()
combined = combined.merge(right=stores_df, on='store_nbr', how='left') \
                    .merge(right=interpolated_oil, on='date', how='left') \
                    .merge(right=final_transactions, on=['date', 'store_nbr'], how='left')

combined['sales'] = np.log1p(combined['sales'])

# 2. Modelling

In [87]:
# FEATURE ENGINEERING
def lag_store_fam(df, time=pd.Timedelta(days=7)):
    curr_df = df.copy()

    shift_df = df[['date', 'sales', 'store_nbr', 'family']].copy()
    shift_df['date'] = shift_df['date'] + time
    shift_df = shift_df.rename(columns={'sales': 'shift_sales'})

    lag_df = pd.merge_asof(left=curr_df, right=shift_df, on='date', by=['store_nbr', 'family'], direction='backward')
    return lag_df.dropna(axis=0)

def date_features(df):
    curr_df = df.copy()
    curr_df['day_of_week'] = df['date'].dt.day_of_week
    curr_df['day_of_year'] = df['date'].dt.day_of_year
    curr_df['day_of_month'] = df['date'].dt.day
    curr_df['month'] = df['date'].dt.month
    return curr_df

def encode(df):
    curr_df = df.copy()
    for col in curr_df.select_dtypes(include='object').columns.difference(['date']):
        curr_df[col], _ = pd.factorize(curr_df[col])
    return curr_df

In [88]:
fe = lag_store_fam(combined)
fe = date_features(fe)
fe = encode(fe)

In [89]:
fe = fe.drop(columns=['city', 'state', 'type', 'cluster'])

In [90]:
fe

,date,store_nbr,family,sales,onpromotion,dcoilwtico,transactions,shift_sales,day_of_week,day_of_year,day_of_month,month
10692,2013-01-09,1,0,1.098612,0,93.08,1910.0,1.098612,2,9,9,1
10693,2013-01-09,1,1,0.000000,0,93.08,1910.0,0.000000,2,9,9,1
10694,2013-01-09,1,2,0.693147,0,93.08,1910.0,1.098612,2,9,9,1
10695,2013-01-09,1,3,7.079184,0,93.08,1910.0,6.995766,2,9,9,1
10696,2013-01-09,1,4,0.000000,0,93.08,1910.0,0.000000,2,9,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2551819,2017-08-14,9,28,5.650484,0,47.59,1971.0,6.131313,0,226,14,8
2551820,2017-08-14,9,29,4.745975,0,47.59,1971.0,4.794964,0,226,14,8
2551821,2017-08-14,9,30,7.207434,7,47.59,1971.0,7.424220,0,226,14,8
2551822,2017-08-14,9,31,5.209486,11,47.59,1971.0,4.990433,0,226,14,8


In [91]:
model = HistGradientBoostingRegressor()

mean_squared_errors = []
for family in fe['family'].unique():
    filtered = fe[fe['family'] == family].drop(columns='family')

    train = filtered[filtered['date'] < '2017-01-01'].drop(columns='date')
    test = filtered[filtered['date'] >= '2017-01-01'].drop(columns='date')

    X_train, y_train = train.drop(columns='sales'), train['sales']
    X_test, y_test = test.drop(columns='sales'), test['sales']

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = root_mean_squared_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    mean_squared_errors.append(mse)

    print(family, f': {rmse:.5f}')

error = np.sqrt(np.mean(mean_squared_errors))
print(f'Final Error: {error:.5f}')

0 : 0.52874
1 : 0.32149
2 : 0.50266
3 : 0.26546
4 : 0.45152
5 : 0.17145
6 : 0.66502
7 : 0.22863
8 : 0.15759
9 : 0.18213
10 : 0.32186
11 : 0.30083
12 : 0.16269
13 : 0.60522
14 : 0.55827
15 : 0.57182
16 : 0.46318
17 : 0.41880
18 : 0.44064
19 : 0.59280
20 : 0.76844
21 : 0.66555
22 : 0.92300
23 : 0.69411
24 : 0.22974
25 : 0.22101
26 : 0.67603
27 : 0.66746
28 : 0.22781
29 : 0.32445
30 : 0.64780
31 : 0.66769
32 : 0.52324
Final Error: 0.50213
